# 💬 Chat-Commerce 🛒

## Setup

In [ ]:
# Install packages
%pip install --upgrade python-dotenv langchain chromadb

In [ ]:
from dotenv import load_dotenv

# Load environment values
load_dotenv()

In [ ]:
import logging
import sys

# Turn off unwanted logs
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Index E-Commerce Data

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

loader = CSVLoader('./data/home-and-garden.csv', csv_args={})
documents = loader.load()

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(documents, embeddings, persist_directory="./index/")

## Run Queries

In [ ]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

system_prompt = PromptTemplate(
    template="""
You are a shopping assistant that helps visitors find the right product from the store.
If you can't find the answer from the context, ask more question. Do not make
Once you have an answer, list down the products in a clear tone as markdown list.
----------------
{context}""",
    input_variables=["context"],
)

system_message_prompt = SystemMessagePromptTemplate(prompt=system_prompt)

human_message_prompt = HumanMessagePromptTemplate.from_template("{question}")

chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt],
)

In [ ]:
from langchain import VectorDBQA
from langchain.agents import Tool

product_search = VectorDBQA.from_chain_type(
    llm=chat, chain_type="stuff", vectorstore=docsearch
)

tools = [
    Tool(
        name="Product Search",
        func=product_search.run,
        description="useful for when you need to answer find products available in the store",
    ),
]

In [ ]:
from langchain.agents import initialize_agent
from langchain.memory import ConversationBufferMemory

prefix_default = """
Assistant is a large language model trained by OpenAI.
Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.
Assistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. 
Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descriptions on a wide range of topics.
Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.
"""

prefix = """
You are an AI Shopping Assistant for ShopHome.com, designed to be able to assist the user find the right product in an online store. The store contains products
for various categories such as furnitures, home decors, gardening tools and equipments.
You are given the following filtered products in a shop and a conversation. You should try better to understand the user needs and suggest one or more products. 
Provide a conversational answer based on the products provided. If you have more than one product to recommend, show them as bulleted list.
If you can't find the answer in the context below, say politely "Hmm, I'm not sure." Don't try to make up a product which is not.
"""

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

ask_ai = initialize_agent(
    tools=tools,
    llm=chat,
    agent="chat-conversational-react-description",
    verbose=True,
    memory=memory,
    prefix=prefix,
    input_variables=["input", "agent_scratchpad"],
)


In [ ]:
ask_ai.run("Hey! Can you help me to setup my garden?")

In [ ]:
ask_ai.run("Yes. I need to pots to grow my plants")

In [ ]:
ask_ai.run("Suggest me a pot which is environmentally friendly")

In [ ]:
import IPython

while True:
    request = input("Input: ")
    IPython.display.display(IPython.display.Markdown("User: " + request))

    if request == "Bye" or request == "bye":
        IPython.display.display(IPython.display.Markdown("Bot: Bye!"))
        break
    else:
        response = ask_ai.run(request)
        IPython.display.display(IPython.display.Markdown("Bot: " + response))


In [ ]:
memory.buffer